In [9]:
import os
import re
from nltk.tokenize import TreebankWordTokenizer
import pandas as pd

from nltk.stem import WordNetLemmatizer

In [32]:
ready_df = pd.DataFrame()

In [4]:
def preprocess(text):

    tokenized_text = TreebankWordTokenizer().tokenize(text)
    
    wordsFiltered = []

    for w in tokenized_text:
        for wf in re.findall(r'(?u)(\w+)', w):
            #for wfreal in re.findall(r'(\D+)', wf):
            wordsFiltered.append(''.join(wf))
    
    return wordsFiltered

In [46]:
def get_context(file_path):
    
    with open(file_path) as fl:
        try:
            text = fl.read()
        except:
            print ("Error: " + file_path )
    
    reg_term = re.compile(r'&(.*?)!&')
    indexes = []
    for i in reg_term.finditer(text):
        indexes.append((i.start(), i.end()))

    left_context = []
    right_context = []
    left_word = []

    for ind in indexes:
        left_part = text[:ind[0]]
        right_part = text[ind[1]+1:]
        clear_left = preprocess(left_part)
        clear_right = preprocess(right_part)
    
    
        #left context
        if len(clear_left) > 5:
            left_context.append(clear_left[-5:])
            left_word.append(clear_left[-1])
        else:
            left_context.append(clear_left)
            if len(clear_left) > 0:
                left_word.append(clear_left[-1])
            else:
                left_word.append(' ')
    
        #right context
        if len(clear_right) > 5:
            right_context.append(clear_right[:5])
        else:
            right_context.append(clear_right)
            
    
    terms = reg_term.findall(text)
    
    global ready_df
    lemmatizer = WordNetLemmatizer()
    
    
    for i in range(len(terms)):
        lft = left_context[i]
        lw = str(left_word[i])
        term = terms[i]
        rgt = right_context[i]
        lc = ' '.join(lft)
        rc = ' '.join(rgt)
        
        df = pd.DataFrame([[lc, lw, term, rc]],
                      columns=['left_context', 'left_word', 'term', 'right_context'])
        ready_df = ready_df.append(df)


In [47]:
start_path = './terms_markup/Marked/'

for path, dirs, files in os.walk(start_path):

    for fname in files:
        if not fname.startswith('.'):
            get_context(start_path+fname)


In [48]:
len(ready_df)

2266

In [55]:
#ready_df.tail(500)

In [52]:
df_sorted = ready_df.sort_values(['left_word'])

In [57]:
df_sorted.tail()

,left_context,left_word,term,right_context
0,силой противящейся движению поезда,является,аэродинамическое сопротивление,Предыдущий рекорд скорости был установлен
0,огромной опорой именно на язык,язык,эмоциональной рефлексии,об этом пишет польско австралийский
0,огромной опорой именно на,язык,эмоциональной рефлексии,об этом пишет польско австралийский
0,на так называемые внешнеполинезийские языки,языки,носители,которых живут за пределами Полинезийского
0,на так называемые внешнеполинезийские,языки,носители,которых живут за пределами Полинезийского


In [25]:
ready_df.to_csv('terms_context.csv', sep='\t')

In [54]:
df_sorted.to_csv('terms_sorted.csv', sep = '\t')